In [105]:
# -*- coding: utf-8 -*-
import models
import itertools

# 4 seems to be the general idea. Could be different. Need to find out
d = distortion_limit = 4

# Start working on a single sentence generalize later
sentence_orig = u"honorables sénateurs , que se est - il passé ici , mardi dernier ?"
sentence = sentence_orig.split(' ')

# Get a translation for each set of words using tm
tm = models.TM("../data/tm", 1)

# We need to find all combinations of the input sentence, and then check if a combination exists in the
# translation model. Earlier implementation had a search scheme going on, which i) did not check for all
# combinations, ii) was highly inefficient since it was searching through the dictionary, which is an O(n)
# operation. Indexing a dictionary is O(1).

# define script P, set of all possible phrases in a sentence
sc_P1 = []

for L in range(1, len(sentence)+1):
    for subset in itertools.combinations(sentence, L):
        # to ensure we preserve the order of combination, we check if the combination 
        # actually appears in the sentence
        if u' '.join(subset) in sentence_orig:
            try:
                print(tm[subset])
                print(subset)
                sc_P1.append((subset, tm[subset]))
            except KeyError:
                continue

[phrase(english='honourable', logprob=0.0)]
(u'honorables',)
[phrase(english=',', logprob=-0.0358480773866)]
(u',',)
[phrase(english='that', logprob=-0.154812589288)]
(u'que',)
[phrase(english='is', logprob=-0.628903687)]
(u'se',)
[phrase(english='is', logprob=-0.0662267655134)]
(u'est',)
[phrase(english='-', logprob=-0.0898769646883)]
(u'-',)
[phrase(english='it', logprob=-0.396981835365)]
(u'il',)
[phrase(english='here', logprob=-0.179900884628)]
(u'ici',)
[phrase(english=',', logprob=-0.0358480773866)]
(u',',)
[phrase(english='Tuesday', logprob=-0.00599036365747)]
(u'mardi',)
[phrase(english='last', logprob=-0.129672244191)]
(u'dernier',)
[phrase(english='?', logprob=-0.000198262714548)]
(u'?',)
[phrase(english=', that', logprob=-0.195274576545)]
(u',', u'que')
[phrase(english='that if', logprob=-0.412180453539)]
(u'que', u'se')
[phrase(english='has', logprob=-0.39545121789)]
(u'se', u'est')
[phrase(english='is', logprob=-0.307978868484)]
(u'est', u'-')
[phrase(english='is it', logp

Reading translation model from ../data/tm...


In [5]:
sentence_orig

u'honorables s\xe9nateurs , que se est - il pass\xe9 ici , mardi dernier ?'

In [7]:
list(enumerate(sentence_orig.split(' ')))

[(0, u'honorables'),
 (1, u's\xe9nateurs'),
 (2, u','),
 (3, u'que'),
 (4, u'se'),
 (5, u'est'),
 (6, u'-'),
 (7, u'il'),
 (8, u'pass\xe9'),
 (9, u'ici'),
 (10, u','),
 (11, u'mardi'),
 (12, u'dernier'),
 (13, u'?')]

In [95]:
for t in range(len(indices)-1):
    print(t)

0
1
2
3


In [94]:
indices

[9, 10, 11, 12, 13]

In [93]:
len(indices)

5

In [101]:
for subset in itertools.combinations(list(enumerate(sentence_orig.split(' '))), 3):
    indices = [x[0] for x in subset]
    flag = 0
    for t in range(len(indices)-1):
        if indices[t+1] - indices[t] > 1:
            flag = 1
    if flag == 0:
        print(indices)
        print(subset)
        print('\n')

[0, 1, 2]
((0, u'honorables'), (1, u's\xe9nateurs'), (2, u','))


[1, 2, 3]
((1, u's\xe9nateurs'), (2, u','), (3, u'que'))


[2, 3, 4]
((2, u','), (3, u'que'), (4, u'se'))


[3, 4, 5]
((3, u'que'), (4, u'se'), (5, u'est'))


[4, 5, 6]
((4, u'se'), (5, u'est'), (6, u'-'))


[5, 6, 7]
((5, u'est'), (6, u'-'), (7, u'il'))


[6, 7, 8]
((6, u'-'), (7, u'il'), (8, u'pass\xe9'))


[7, 8, 9]
((7, u'il'), (8, u'pass\xe9'), (9, u'ici'))


[8, 9, 10]
((8, u'pass\xe9'), (9, u'ici'), (10, u','))


[9, 10, 11]
((9, u'ici'), (10, u','), (11, u'mardi'))


[10, 11, 12]
((10, u','), (11, u'mardi'), (12, u'dernier'))


[11, 12, 13]
((11, u'mardi'), (12, u'dernier'), (13, u'?'))




## Using a simple implementation

In [133]:
sentence

[u'honorables',
 u's\xe9nateurs',
 u',',
 u'que',
 u'se',
 u'est',
 u'-',
 u'il',
 u'pass\xe9',
 u'ici',
 u',',
 u'mardi',
 u'dernier',
 u'?']

In [137]:
sentence[2:2]

[]

In [ ]:
2, 4

In [135]:
sentence[0], sentence[1:2+1]

(u'honorables', [u's\xe9nateurs'])

In [138]:
sc_P = []

for i in range(0, len(sentence)):
    for j in range(i, len(sentence)):
        if i == j:
            subset = (sentence[i],)
        else:
            subset = tuple(sentence[i:j+1])
        print(i, j)
        print(subset)
        try:
            tm_output = tm[subset]
            sc_P.append((i, j, tm_output))
        except KeyError:
            continue
        

(0, 0)
(u'honorables',)
(0, 1)
(u'honorables', u's\xe9nateurs')
(0, 2)
(u'honorables', u's\xe9nateurs', u',')
(0, 3)
(u'honorables', u's\xe9nateurs', u',', u'que')
(0, 4)
(u'honorables', u's\xe9nateurs', u',', u'que', u'se')
(0, 5)
(u'honorables', u's\xe9nateurs', u',', u'que', u'se', u'est')
(0, 6)
(u'honorables', u's\xe9nateurs', u',', u'que', u'se', u'est', u'-')
(0, 7)
(u'honorables', u's\xe9nateurs', u',', u'que', u'se', u'est', u'-', u'il')
(0, 8)
(u'honorables', u's\xe9nateurs', u',', u'que', u'se', u'est', u'-', u'il', u'pass\xe9')
(0, 9)
(u'honorables', u's\xe9nateurs', u',', u'que', u'se', u'est', u'-', u'il', u'pass\xe9', u'ici')
(0, 10)
(u'honorables', u's\xe9nateurs', u',', u'que', u'se', u'est', u'-', u'il', u'pass\xe9', u'ici', u',')
(0, 11)
(u'honorables', u's\xe9nateurs', u',', u'que', u'se', u'est', u'-', u'il', u'pass\xe9', u'ici', u',', u'mardi')
(0, 12)
(u'honorables', u's\xe9nateurs', u',', u'que', u'se', u'est', u'-', u'il', u'pass\xe9', u'ici', u',', u'mardi', u

In [141]:
len(sc_P)

27

In [139]:
sc_P

[(0, 0, [phrase(english='honourable', logprob=0.0)]),
 (2, 2, [phrase(english=',', logprob=-0.0358480773866)]),
 (2, 3, [phrase(english=', that', logprob=-0.195274576545)]),
 (2, 4, [phrase(english=', that if', logprob=0.0)]),
 (2, 7, [phrase(english=', what', logprob=0.0)]),
 (3, 3, [phrase(english='that', logprob=-0.154812589288)]),
 (3, 4, [phrase(english='that if', logprob=-0.412180453539)]),
 (3, 5, [phrase(english='what has', logprob=-0.301030009985)]),
 (3, 7, [phrase(english='what', logprob=0.0)]),
 (4, 4, [phrase(english='is', logprob=-0.628903687)]),
 (4, 5, [phrase(english='has', logprob=-0.39545121789)]),
 (4, 6, [phrase(english='did', logprob=0.0)]),
 (4, 7, [phrase(english='come he did', logprob=-0.301030009985)]),
 (5, 5, [phrase(english='is', logprob=-0.0662267655134)]),
 (5, 6, [phrase(english='is', logprob=-0.307978868484)]),
 (5, 7, [phrase(english='is it', logprob=-0.219359949231)]),
 (6, 6, [phrase(english='-', logprob=-0.0898769646883)]),
 (6, 7, [phrase(english='

In [140]:
sc_P_indices

[(0, 0, [phrase(english='honourable', logprob=0.0)]),
 (2, 2, [phrase(english=',', logprob=-0.0358480773866)]),
 (3, 3, [phrase(english='that', logprob=-0.154812589288)]),
 (4, 4, [phrase(english='is', logprob=-0.628903687)]),
 (5, 5, [phrase(english='is', logprob=-0.0662267655134)]),
 (6, 6, [phrase(english='-', logprob=-0.0898769646883)]),
 (7, 7, [phrase(english='it', logprob=-0.396981835365)]),
 (9, 9, [phrase(english='here', logprob=-0.179900884628)]),
 (10, 10, [phrase(english=',', logprob=-0.0358480773866)]),
 (11, 11, [phrase(english='Tuesday', logprob=-0.00599036365747)]),
 (12, 12, [phrase(english='last', logprob=-0.129672244191)]),
 (13, 13, [phrase(english='?', logprob=-0.000198262714548)]),
 (2, 3, [phrase(english=', that', logprob=-0.195274576545)]),
 (3, 4, [phrase(english='that if', logprob=-0.412180453539)]),
 (4, 5, [phrase(english='has', logprob=-0.39545121789)]),
 (5, 6, [phrase(english='is', logprob=-0.307978868484)]),
 (6, 7, [phrase(english='is it', logprob=-0.32

In [116]:
# define script P, set of all possible phrases in a sentence
sc_P = []

for L in range(1, len(sentence)+1):
    for subset in itertools.combinations(list(enumerate(sentence_orig.split(' '))), L):
        # to ensure we preserve the order of combination, we check if the combination 
        # actually appears in the sentence
            indices = [x[0] for x in subset]
            flag = 0
            # first, the subset should be in the sentence
            if u' '.join(tuple([x[1] for x in subset])) in sentence_orig:
                # then we ensure that the chosen subset is actually adjacent. 
                for t in range(len(indices)-1):
                    if indices[t+1] - indices[t] > 1:
                        flag = 1
                if flag == 0:
                    try:
#                         print(tm[tuple([x[1] for x in subset])])
#                         print(subset)
                        sc_P.append((subset, tm[tuple([x[1] for x in subset])]))
                    except KeyError:
                        continue
                        
sc_P_indices = []
ind = 0 # index is at zero place
for inst in range(len(sc_P)):
    indices = []
    for source_phrase in sc_P[inst][0]: # source phrase is at pos 0
        indices.append(source_phrase[ind])
    start = min(indices)
    end = max(indices)
    sc_P_indices.append((start, end, sc_P[inst][1]))

In [117]:
sc_P_indices

[(0, 0, [phrase(english='honourable', logprob=0.0)]),
 (2, 2, [phrase(english=',', logprob=-0.0358480773866)]),
 (3, 3, [phrase(english='that', logprob=-0.154812589288)]),
 (4, 4, [phrase(english='is', logprob=-0.628903687)]),
 (5, 5, [phrase(english='is', logprob=-0.0662267655134)]),
 (6, 6, [phrase(english='-', logprob=-0.0898769646883)]),
 (7, 7, [phrase(english='it', logprob=-0.396981835365)]),
 (9, 9, [phrase(english='here', logprob=-0.179900884628)]),
 (10, 10, [phrase(english=',', logprob=-0.0358480773866)]),
 (11, 11, [phrase(english='Tuesday', logprob=-0.00599036365747)]),
 (12, 12, [phrase(english='last', logprob=-0.129672244191)]),
 (13, 13, [phrase(english='?', logprob=-0.000198262714548)]),
 (2, 3, [phrase(english=', that', logprob=-0.195274576545)]),
 (3, 4, [phrase(english='that if', logprob=-0.412180453539)]),
 (4, 5, [phrase(english='has', logprob=-0.39545121789)]),
 (5, 6, [phrase(english='is', logprob=-0.307978868484)]),
 (6, 7, [phrase(english='is it', logprob=-0.32

In [118]:
list(enumerate(sentence_orig.split(' ')))

[(0, u'honorables'),
 (1, u's\xe9nateurs'),
 (2, u','),
 (3, u'que'),
 (4, u'se'),
 (5, u'est'),
 (6, u'-'),
 (7, u'il'),
 (8, u'pass\xe9'),
 (9, u'ici'),
 (10, u','),
 (11, u'mardi'),
 (12, u'dernier'),
 (13, u'?')]

In [110]:
for i in range(len(sc_P)):
    print(sc_P[i])
    print('-'*50)
    print(sc_P1[i])
    print('\n')

(((0, u'honorables'),), [phrase(english='honourable', logprob=0.0)])
--------------------------------------------------
((u'honorables',), [phrase(english='honourable', logprob=0.0)])


(((2, u','),), [phrase(english=',', logprob=-0.0358480773866)])
--------------------------------------------------
((u',',), [phrase(english=',', logprob=-0.0358480773866)])


(((3, u'que'),), [phrase(english='that', logprob=-0.154812589288)])
--------------------------------------------------
((u'que',), [phrase(english='that', logprob=-0.154812589288)])


(((4, u'se'),), [phrase(english='is', logprob=-0.628903687)])
--------------------------------------------------
((u'se',), [phrase(english='is', logprob=-0.628903687)])


(((5, u'est'),), [phrase(english='is', logprob=-0.0662267655134)])
--------------------------------------------------
((u'est',), [phrase(english='is', logprob=-0.0662267655134)])


(((6, u'-'),), [phrase(english='-', logprob=-0.0898769646883)])
------------------------------------

In [55]:
len(sc_P[inst][source_phrase])

2

In [54]:
inst = 15
source_phrase = 0
curr_word_ind = 0
curr_word = 0

print(sc_P[inst])
print(sc_P[inst][source_phrase])
print(sc_P[inst][source_phrase][curr_word])
print(sc_P[inst][source_phrase][curr_word][ind])

(((5, u'est'), (6, u'-')), [phrase(english='is', logprob=-0.307978868484)])
((5, u'est'), (6, u'-'))
(5, u'est')
5


In [77]:
for source_phrase in sc_P[15][0]:
    print(source_phrase)

(5, u'est')
(6, u'-')


In [111]:
sc_P_indices = []
ind = 0 # index is at zero place
for inst in range(len(sc_P)):
    indices = []
    for source_phrase in sc_P[inst][0]: # source phrase is at pos 0
        indices.append(source_phrase[ind])
    start = min(indices)
    end = max(indices)
    sc_P_indices.append((start, end, sc_P[inst][1]))

In [113]:
for i in range(len(sc_P)):
    print(sc_P[i])
    print('-'*50)
    print(sc_P_indices[i])
    print('\n')

(((0, u'honorables'),), [phrase(english='honourable', logprob=0.0)])
--------------------------------------------------
(0, 0, [phrase(english='honourable', logprob=0.0)])


(((2, u','),), [phrase(english=',', logprob=-0.0358480773866)])
--------------------------------------------------
(2, 2, [phrase(english=',', logprob=-0.0358480773866)])


(((3, u'que'),), [phrase(english='that', logprob=-0.154812589288)])
--------------------------------------------------
(3, 3, [phrase(english='that', logprob=-0.154812589288)])


(((4, u'se'),), [phrase(english='is', logprob=-0.628903687)])
--------------------------------------------------
(4, 4, [phrase(english='is', logprob=-0.628903687)])


(((5, u'est'),), [phrase(english='is', logprob=-0.0662267655134)])
--------------------------------------------------
(5, 5, [phrase(english='is', logprob=-0.0662267655134)])


(((6, u'-'),), [phrase(english='-', logprob=-0.0898769646883)])
--------------------------------------------------
(6, 6, [phrase

In [81]:
list(enumerate(sentence_orig.split(' ')))

[(0, u'honorables'),
 (1, u's\xe9nateurs'),
 (2, u','),
 (3, u'que'),
 (4, u'se'),
 (5, u'est'),
 (6, u'-'),
 (7, u'il'),
 (8, u'pass\xe9'),
 (9, u'ici'),
 (10, u','),
 (11, u'mardi'),
 (12, u'dernier'),
 (13, u'?')]

In [83]:
sc_P[10]

(((12, u'dernier'),), [phrase(english='last', logprob=-0.129672244191)])

In [84]:
sc_P_indices[10]

(12, 12, [phrase(english='last', logprob=-0.129672244191)])

In [58]:
sc_P[1][1]

[phrase(english=',', logprob=-0.0358480773866)]